<a href="https://colab.research.google.com/github/gauss5930/Huggingface-Course/blob/main/Chapter%207./Fine_tuning_a_masked_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a masked language model

## MLM을 위한 pretrained 모델 선택

MLM을 진행하기 위해 적합한 pretrained model을 선택한다. Hugging Face Hub에서 'Fill-Mask' 필터를 사용해 손쉽게 후보자를 찾을 수 있다.

이번에는 DistilBERT를 사용해보도록 하겠다. 그러기 위해 AutoModelFOrMaskedLM 클래스를 사용해서 DistilBERT를 다운로드 해보도록 하자.



In [1]:
!pip install transformers
from transformers import AutoModelForMaskedLM

model_checkpoint = 'distilbert-base-uncased'
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.8 MB/s eta 0:00:00


DistilBERT가 예측해야 하는 작은 텍스트 샘플을 봐보도록 하자. 사람 같은 경우에는 [MASK] 토큰에 대해서 'day', 'ride', 'painting' 같은 많은 경우의 수를 생각해낼 수 있다. pretrained model에 대해 예측은 모델이 학습한 corpus에 의존하기 때문에 데이터에서 나타나는 통계적 패턴에 의해 학습된다.

In [2]:
text = 'This is a great [MASK].'

mask를 예측하기 위해 모델에 대한 입력을 생성하기 위한 DistilBERT의 tokenizer가 필요하다. 이 tokenizer을 Hub에서 다운로드 받아보도록 하자.

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer와 모델을 사용하여 텍스트 예시를 모델에게 줘서 logit을 이끌어 내고, top-5 후보자를 출력해낼 수 있다.

In [4]:
import torch

inputs = tokenizer(text, return_tensors = 'pt')
token_logits = model(**inputs).logits
# [MASK]의 위치를 찾아내고 이것의 logit을 추출
mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# 높은 logit 값을 사용하여 [MASK] 후보자 선정
top_5_tokens = torch.topk(mask_token_logits, 5, dim = 1).indices[0].tolist()

for token in top_5_tokens:
  print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

'>>> This is a great deal.
'>>> This is a great success.
'>>> This is a great adventure.
'>>> This is a great idea.
'>>> This is a great feat.


위와 같이 모델이 학습된 데이터셋인 English Wikipedia를 따라서 괜찮은 대답을 내놓는 것을 볼 수 있다. 그렇다면 이제 도메인을 '영화 리뷰'로 바꾸는 방법에 대해 알아보자!

## The dataset

도메인 적응을 위해 영화 리뷰의 corpus인 유명한 데이터셋 'Large Movie Review Dataset(IMDb)'을 사용할 것이다. DistilBERT를 이 corpus에서 fine-tune 함으로써, LM이 영화 리뷰의 보다 주관적인 요소에 대해 pre-train된 Wikipedia의 사실적 데이터에서 vocabulary를 적응시킬 것으로 기대된다. 데이터셋은 Hugging Face Hub로부터 load_dataset() 함수를 사용해서 불러올 수 있다.

In [5]:
!pip install datasets
from datasets import load_dataset

imdb_dataset = load_dataset('imdb')
imdb_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.8 MB/s eta 0:00:00


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

## Preprocessing the data

전처리를 위해 처음에 corpus를 평범하게 토큰화하지만 우리의 tokenizer에서 truncation=True 옵션 없이 세팅한다. 또한 word ID가 사용 가능한 경우 word ID를 가져온다. 나중에 전체 단어 마스킹을 수행하는 데 필요하다. 이를 간단한 함수로 감싸고 더 이상 필요하지 않으므로 텍스트 및 라벨 열을 제거한다.

In [6]:
def tokenize_function(examples):
  result = tokenizer(examples['text'])
  if tokenizer.is_fast:
    result['word_ids'] = [result.word_ids(i) for i in range(len(result['input_ids']))]
  return result

# batched = True를 사용해서 fast multithreading을 활성화한다!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched = True, remove_columns = ['text', 'label']
)
tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

이렇게 해서 모든 영화 리뷰는 토큰화되었고, 다음 스텝은 모든 것을 함께 그룹화하고 결과를 chunk로 분할하는 것이다. 하지만 이 chunk는 얼마나 커야하는 걸까? 이것은 GPU 사용 가능량에 따라 달라지는데 보통은 model_max_length 특성으로 설정한다.

In [7]:
tokenizer.model_max_length

512

Colab의 GPU를 사용해서 실험을 수행하기 위해서는 메모리에 적용될 수 있는 조금 작은 값을 선택한다.

In [8]:
chunk_size = 128

이 모든 걸 종합해서 토큰화된 데이터셋에 적용할 수 있는 하나의 함수를 만든다.

In [9]:
def group_texts(examples):
  # 모든 텍스트를 연결
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  # 연결된 텍스트의 길이를 계산
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  # 마지막 chunk가 chunk_size보다 작으면 drop
  total_length = (total_length // chunk_size) * chunk_size
  # max_len의 chunk로 분할
  result = {
      k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
      for k, t in concatenated_examples.items()
  }
  # 새로운 라벨 열을 생성
  result['labels'] = result['input_ids'].copy()
  return result

group_texts 함수의 마지막 부분을 보면 input_ids를 복사해서 새로운 label 열을 만든 것을 알 수 있다. 이는 MLM에서 objective가 입력 배치에서 랜덤하게 마스킹된 토큰을 예측하는 것이고, label 열을 만듦으로써 이로부터 학습하기 위한 LM에 대한 ground truth를 제공한다.

이제 신뢰할 수 있는 Dataset.map() 함수를 사용해서 토큰화된 데이터셋에 group_texts()를 적용해보자. 

In [10]:
lm_datasets = tokenized_datasets.map(group_texts, batched = True)
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

## Trainer API를 사용해서 DistilBERT Fine-tuning

MLM을 fine-tune 하는 것은 문장 분류를 하는 것과 거의 유사하다. 유일한 차이점은 각 텍스트 배치에서 몇 개의 토큰을 랜덤하게 마스킹하는 special data collator가 필요하다는 것이다. 이것은 🤗 Transformers의 DataCollatorForLanguageModeling으로 해결이 가능하다. 마스킹할 토큰의 비율을 지정하는 tokznier와 mlm_probability 인수를 전달하기만 하면 된다. BERT에 사용되는 양이며 문헌에서 일반적으로 선택되는 15%를 선택한다.

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15)

MLM을 위한 모델을 학습시킬 때, 전체 단어를 함께 마스킹하는데 사용되는 하나의 기술이 사용될 수 있다. 이 whole word masking을 사용하기 위해서는 data collator을 만들 필요가 있다. data collator는 샘플 리스트를 받아서 배치로 변환하는 하나의 함수이므로, 만들어보도록 하자!

In [12]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2

def whole_word_masking_data_collator(features):
  for feature in features:
    word_ids = feature.pop('word_ids')

    # 단어와 해당 토큰 인덱스들 간의 맵 생성
    mapping = collections.defaultdict(list)
    current_word_index = -1
    current_word = None
    for idx, word_id in enumerate(word_ids):
      if word_id is not None:
        if word_id != current_word:
          current_word = word_id
          current_word_index += 1
        mapping[current_word_index].append(idx)

    # 랜덤하게 단어를 마스킹
    mask = np.random.binomial(1, wwm_probability, (len(mapping),))
    input_ids = feature['input_ids']
    labels = feature['labels']
    new_labels = [-100] * len(labels)
    for word_id in np.where(mask)[0]:
      word_id = word_id.item()
      for idx in mapping[word_id]:
        new_labels[idx] = labels[idx]
        input_ids[idx] = tokenizer.mask_token_id
    feature['labels'] = new_labels

  return default_data_collator(features)

In [13]:
# 똑같은 샘플에 시도해보자!
samples = [lm_datasets['train'][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch['input_ids']:
  print(f"\n'>>> {tokenizer.decode(chunk)}")


'>>> [CLS] i rented [MASK] am [MASK] [MASK] yellow from my video store [MASK] of all the controversy that surrounded it when it [MASK] first released in 1967. i [MASK] heard that at first it was seized by u. s. customs if it ever tried to enter this country, therefore being a [MASK] of films considered " controversial " i really had to [MASK] [MASK] [MASK] myself. < br [MASK] [MASK] < br / > the plot is centered [MASK] a young swedish drama student named [MASK] who wants to learn everything [MASK] can [MASK] life. in particular [MASK] wants to focus her attentions to making some sort of documentary on [MASK] the average [MASK] [MASK] [MASK] about certain [MASK] issues such

'>>> as the vietnam [MASK] and [MASK] issues in the [MASK] states. in [MASK] asking [MASK] and ordinary denizens of stockholm about their opinions on politics, she has sex with [MASK] [MASK] teacher, classmates, and married men [MASK] < br / [MASK] < br / > what kills me about i am curious - yellow is that 40 years

이제 남은 fine-tuning step은 일반적이지만, 만약 Google Colab의 P100GPU를 사용하지 못한다면, training set의 사이즈를 다운 샘플링해서 몇 천개의 example로 만들어야 한다. training set의 사이즈가 줄어든다고 해도 걱정하지 마라! 그래도 충분히 훌륭한 LM이니까! 데이터셋을 다운 샘플링하는 간단한 방법은 🤗 Datasets에서 Dataset.train_test_split()을 사용하는 것이다.

In [14]:
train_size = 10000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets['train'].train_test_split(
    train_size = train_size, test_size = test_size, seed = 42
)

이제 해야할 것은 Hugging Face Hub에 로그인하는 것이다.

In [15]:
from huggingface_hub import notebook_login

notebook_login()

로그인 되었으면 Trainer에 대한 argument를 명시할 수 있다.

In [16]:
!pip install --upgrade accelerate
from transformers import TrainingArguments

batch_size = 64
# 매 에폭마다 training loss 보여주기
logging_steps = len(downsampled_dataset['train']) // batch_size
model_name = model_checkpoint.split('/')[-1]

training_args = TrainingArguments(
    output_dir = f"{model_name}-finetuned-imdb",
    overwrite_output_dir = True,
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    weight_decay = 0.01,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    push_to_hub = True,
    fp16 = True,
    logging_steps = logging_steps,
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 18.2 MB/s eta 0:00:00


Trainer를 초기화하기 위한 모든 요소가 갖춰졌다. 여기서는 기존의 data_collator를 사용하지만, 결과를 비교하기 위해 whole word masking collator를 사용할 수도 있다.

In [18]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = downsampled_dataset['train'],
    eval_dataset = downsampled_dataset['test'],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

Cloning https://huggingface.co/Cartinoe5930/distilbert-base-uncased-finetuned-imdb into local empty directory.


이제 trainer.train()을 수행하기 위한 모든 준비가 끝났지만, 이를 하기 전에 *perplexity*에 대해 간략하게 살펴보도록 하자. 이 metric은 LM의 성능을 평가하기 위한 간단한 metric이다.

### Perplexity for language models

테스트 세트가 대부분 문법적으로 올바른 문장으로 구성되어 있다고 가정하면 언어 모델의 품질을 측정하는 한 가지 방법은 테스트 세트의 모든 문장에서 다음 단어에 할당하는 확률을 계산하는 것입니다. 확률이 높다는 것은 모델이 보이지 않는 예에 "놀라거나" "당황"하지 않는다는 것을 나타내며 언어의 기본 문법 패턴을 학습했음을 시사합니다. perplexity의 수학적 정의에는 매우 다양한 것들이 있지만, 우리가 사용할 것은 cross-entropy loss의 제곱이다. 따라서 테스트 세트에서 cross-entropy loss를 걔산하고 그 결과에 제곱을 하는 Trainer.evaluate() 함수를 사용함으로써 우리의 pre-trained LM의 perplexity를 계산할 수 있다.

In [19]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


>>> Perplexity: 21.94


낮은 perplexity 점수는 더 나은 LM을 의미하고, 초기 모델은 어느 정도 큰 값을 가진다. 이를 fine-tuning을 통해 줄여보도록 하자! 이를 하기 위해서 training loop를 돌려보도록 하자.

In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.708600,2.489690
2,2.579600,2.422971
3,2.526900,2.435377


TrainOutput(global_step=471, training_loss=2.604786610653952, metrics={'train_runtime': 154.5284, 'train_samples_per_second': 194.139, 'train_steps_per_second': 3.048, 'total_flos': 994208670720000.0, 'train_loss': 2.604786610653952, 'epoch': 3.0})

그 다음에 테스트 세트에서의 perplexity를 계산해보자.

In [21]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 11.85


이제 모델 카드를 Hub에 업데이트 해보도록 하자.

In [22]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/256M [00:00<?, ?B/s]

Upload file runs/May19_13-09-09_355ff7864fd5/events.out.tfevents.1684501946.355ff7864fd5.586.2:   0%|         …

Upload file runs/May19_13-09-09_355ff7864fd5/events.out.tfevents.1684501790.355ff7864fd5.586.0:   0%|         …

Upload file training_args.bin:   0%|          | 1.00/3.87k [00:00<?, ?B/s]

Upload file runs/May19_13-09-09_355ff7864fd5/1684501790.2465613/events.out.tfevents.1684501790.355ff7864fd5.58…

To https://huggingface.co/Cartinoe5930/distilbert-base-uncased-finetuned-imdb
   e657258..089981d  main -> main

   e657258..089981d  main -> main

To https://huggingface.co/Cartinoe5930/distilbert-base-uncased-finetuned-imdb
   089981d..0439a57  main -> main

   089981d..0439a57  main -> main



'https://huggingface.co/Cartinoe5930/distilbert-base-uncased-finetuned-imdb/commit/089981d3222c71bd35e88e5336e8517e2214112f'

## Using our fine-tuned model

Hub에서 위젯을 사용하거나 🤗 Transformers의 pipeline을 사용하여 로컬에서 fine-tune된 모델과 상호 작용할 수 있다. 후자를 사용하여 fill-mask pipeline을 사용하여 모델을 다운로드해 보겠다.

In [25]:
from transformers import pipeline

mask_filter = pipeline(
    'fill-mask', model = 'Cartinoe5930/distilbert-base-uncased-finetuned-imdb'
)

In [26]:
preds = mask_filter(text)

for pred in preds:
  print(f">>> {pred['sequence']}")

>>> this is a great film.
>>> this is a great movie.
>>> this is a great idea.
>>> this is a great one.
>>> this is a great job.
